In [2]:
# Import necessary libraries
import os
from typing import Any, Dict

import google.generativeai as genai
import requests
from dotenv import load_dotenv

print("We are up and running!")

We are up and running!


Ensure that you created a Gemini API Key and added it to the `.env` file before executing the next command

In [3]:
# Load environment variables
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key or api_key.startswith("ADD YOUR"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

# Configure Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

print(f"Gemini model loaded successfully: {model.model_name}")

Gemini model loaded successfully: models/gemini-2.0-flash


In [4]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(response.text)

I am unable to give you real-time information, including the current price of Bitcoin. Cryptocurrency prices change very, very rapidly.

To find the current price, I recommend checking these resources:

*   **Major Cryptocurrency Exchanges:** Coinbase, Binance, Kraken, Gemini
*   **Financial Websites:** Google Finance, Yahoo Finance, Bloomberg, MarketWatch
*   **Cryptocurrency Tracking Websites:** CoinMarketCap, CoinGecko

These sources will provide you with the most up-to-date price.


In [5]:
#RESPONCE WITH METADTA TYPICAL LLM RESPONCE
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I am unable to give you real-time information, including the current price of Bitcoin. Cryptocurrency prices change very, very rapidly.\n\nTo find the current price, I recommend checking these resources:\n\n*   **Major Cryptocurrency Exchanges:** Coinbase, Binance, Kraken, Gemini\n*   **Financial Websites:** Google Finance, Yahoo Finance, Bloomberg, MarketWatch\n*   **Cryptocurrency Tracking Websites:** CoinMarketCap, CoinGecko\n\nThese sources will provide you with the most up-to-date price."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.1446227372861376
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 8,
        "candidates_token_count": 102,

In [6]:
url = f"https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
response = requests.get(url)
data = response.json()
print(data)

{'symbol': 'BTCUSDT', 'price': '107016.74000000'}


In [7]:

# Define the function
def get_crypto_price(symbol: str) -> Dict[str, Any]:
    """
    Get the current price of a cryptocurrency from Binance API
    """
    url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    return float(data["price"])


In [8]:
price = get_crypto_price("BTCUSDT")
print(f"BTC Price in USDT: {price}")

BTC Price in USDT: 106952.53


In [9]:
tools = [
    {
        "function_declarations": [
            {
                "name": "get_crypto_price",
                "description": "Get cryptocurrency price in USDT from Binance",
                "parameters": {
                    "type": "object", 
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The cryptocurrency trading pair symbol (e.g., BTCUSDT, ETHUSDT). \
                                            The symbol for Bitcoin is BTCUSDT. \
                                            The symbol for Ethereum is ETHUSDT."
                        }
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

In [10]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(response)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "BTCUSDT"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.10051312297582626
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 60,
        "candidates_token_count": 8,
        "total_token_count": 68
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [11]:
price = get_crypto_price("BTCUSDT")

In [12]:
final_response = chat.send_message(str(price))
print(final_response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The current price of Bitcoin (BTC) is $107,051.55.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.029440850019454956
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 25,
        "candidates_token_count": 22,
        "total_token_count": 47
      },
      "model_version": "gemini-2.0-flash"
    }),
)


In [13]:
print(final_response.text)

The current price of Bitcoin (BTC) is $107,051.55.

